In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.nn import Flatten,Linear,ReLU, Conv2d,MaxPool2d

In [16]:
trainset=datasets.MNIST('mnist_dataset',train=True,download=True,transform=transforms.ToTensor())
train_loader=torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True)

In [5]:
model=torch.nn.Sequential(Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=1,padding=1),
                          ReLU(),
                          MaxPool2d(2,2),
                          Conv2d(in_channels=6,out_channels=6,kernel_size=3,stride=1,padding=1),
                          ReLU(),
                          Flatten(),
                          Linear(6*14*14,10)
                          )
model=model.cuda()

optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)

In [18]:
print_every=100
for epoch in range(5):
  print('Epoch: ',epoch)
  totalLoss=0
  totalExamples=0
  totalCorrect=0

  for iterations, (X,y) in enumerate(train_loader):
    X=X.cuda()
    y=y.cuda()

    scores=model(X)

    loss=torch.nn.functional.cross_entropy(scores,y)

    preds=torch.argmax(scores,dim=1)
    correct=torch.sum(preds==y)

    totalLoss+=loss.item()
    totalCorrect+=correct.item()
    totalExamples+=len(X)
    if iterations % print_every==0:
      print('loss=',totalLoss/totalExamples,'correct=',totalCorrect/totalExamples)

    loss.backward()

    with torch.no_grad():
      optimizer.step()
      optimizer.zero_grad()

Epoch:  0
loss= 0.00010290172212990001 correct= 1.0
loss= 0.0004645615715189436 correct= 0.9918007425742574
loss= 0.000516260056270649 correct= 0.9897388059701493
loss= 0.0005045514555719133 correct= 0.9902927740863787
loss= 0.0005207065312520841 correct= 0.9898301122194514
loss= 0.0005298421534596586 correct= 0.9897392714570858
loss= 0.0005358611817334203 correct= 0.9894446755407654
loss= 0.0005626745868995125 correct= 0.9891449714693296
loss= 0.0005594930091873938 correct= 0.9891541822721598
loss= 0.0005601156039509454 correct= 0.9891613485016648
Epoch:  1
loss= 0.0002605376939754933 correct= 1.0
loss= 0.0005340893000207546 correct= 0.9887066831683168
loss= 0.0005223515442562689 correct= 0.9895055970149254
loss= 0.000485991094841744 correct= 0.9899294019933554
loss= 0.0005073158439123821 correct= 0.9894404613466334
loss= 0.0005104563830670401 correct= 0.9896768962075848
loss= 0.0005135713543740668 correct= 0.9895486688851913
loss= 0.0005154701124302093 correct= 0.9896130527817404
los

In [19]:
testset = datasets.MNIST('mnist_dataset', train=False, download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [20]:
correct=0
with torch.no_grad():
  for X,y in test_loader:
    X = X.cuda()
    y = y.cuda()

    scores = model(X)
    preds = torch.argmax(scores, dim=1)

    correct+=torch.sum(preds==y)
  print(correct.item()/len(testset))

0.9857
